In [ ]:
import pm4py
from tokenreplay import TokenReplay
from comparison import ModelComparator
from practical.ProcessMining.group1.shared import utils

In [ ]:
def compare_logs(x_dim="fitness", y_dim="generalization", log_key=None):
    logs = utils.read_txt_test_logs("../shared/example_files/simple_event_logs_modified.txt")
    if log_key:
        logs = {log_key: logs[log_key]}

    for log_key, log in logs.items():
        # print(f"Processing log: {log_key}")
        log_df = utils.event_log_to_dataframe(log)
        # print('log_df =====', log_df)
        log_df = pm4py.format_dataframe(log_df, case_id='case_id', activity_key='activity', timestamp_key='timestamp')

        net_alpha, im_alpha, fm_alpha = pm4py.discover_petri_net_alpha(log_df)

        net_heuristic, im_heuristic, fm_heuristic = pm4py.discover_petri_net_heuristics(log_df)

        net_inductive, im_inductive, fm_inductive = pm4py.discover_petri_net_inductive(log_df)
        
        net_inductive_inf, im_inductive_inf, fm_inductive_inf = pm4py.discover_petri_net_inductive(log_df,noise_threshold=0.7)

        m1 = TokenReplay(log_df, net_alpha, im_alpha, fm_alpha, "AM")
        m2 = TokenReplay(log_df, net_heuristic, im_heuristic, fm_heuristic, "HM")
        m3 = TokenReplay(log_df, net_inductive, im_inductive, fm_inductive, "IM")
        m4 = TokenReplay(log_df, net_inductive_inf, im_inductive_inf, fm_inductive_inf, "IMi")

        comparator = ModelComparator([m1, m2, m3, m4])

        print("===== Model Comparison for log:", log_key, "=====")
        res = comparator.run(x_dimension=x_dim, y_dimension=y_dim)
        print(res)

In [ ]:
compare_logs(log_key="L1")

In [ ]:
compare_logs(x_dim="p", y_dim="s")